<a href="https://colab.research.google.com/github/DantonCoote/My_Hub/blob/DeepLearning/Assign3_Group9_NNDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3 for Neural Networks and Deep Learning

## Step 1.1, Loading the Data

In this assignment we are using a public built in dataset from the Tensorflow/Keras library.<br>
This is a dataset of 50,000 32x32 color training images and 10,000 test images, labeled over 10 categories

In [4]:
import keras as ks


(features_train, label_train), (features_test, label_test) = ks.datasets.cifar10.load_data()

Well things have been loaded. Time to explore our dataset. <br>
There seems to be multiple ways to load this data, one of which doesn't load the set but instead generates it.

## Step 1.2, Data Exploration

In [6]:
# combining data into single dataframes for analysis
import pandas as pd

data_t = pd.DataFrame()
labels_t = label_test + label_train

ValueError: operands could not be broadcast together with shapes (10000,32,32,3) (50000,32,32,3) 

In [5]:
import seaborn as sb
sb.barplot(data=(features_train, features_test), x=(label_test,label_train))

TypeError: Data source must be a DataFrame or Mapping, not <class 'tuple'>.